In [1]:
import pandas as pd
import re
import json
pd.options.mode.chained_assignment = None 

In [2]:
df = pd.read_csv("AN-1068-final-python.csv")

In [3]:
df.head()

,step_id,status,client,uid_anon,id_anon,reply,feedback
0,5205,2,web,4924,2194,"{""language"": ""python3"", ""code"": ""print(\""Learn...","{""message"": ""Passed. Congratulations! It is yo..."
1,5205,2,PyCharmCore,4757,2736,"{""choices"": null, ""score"": null, ""solution"": n...","{""message"": ""Passed. Congratulations! It is yo..."
2,5205,2,web,4925,3664,"{""language"": ""python3"", ""code"": ""print(\""Learn...","{""message"": ""Passed. Congratulations! It is yo..."
3,5205,2,web,4935,5133,"{""language"": ""python3"", ""code"": ""print(\""Learn...","{""message"": ""Passed. Congratulations! It is yo..."
4,5205,2,web,4773,5726,"{""language"": ""python3"", ""code"": ""print(\""Learn...","{""message"": ""Passed. Congratulations! It is yo..."


In [4]:
df.groupby('status')['id_anon'].count()

status
1    2434549
2    2188691
Name: id_anon, dtype: int64

In [5]:
df.groupby('client')['id_anon'].count()

client
Idea              2351
IdeaEdu           3150
PyCharmCore     180123
PyCharmEdu      236225
Python          157684
ide                 53
idea              5356
internal         82729
web            3883207
Name: id_anon, dtype: int64

In [6]:
def label(row):
    if row['status'] == 2:             
        try:
            #print(row['feedback'],'\n')
            json_data = json.loads(row['feedback'],strict=False)   
            if (len(json_data['code_style']['errors'])==0):
                return 'Correct'  
            else:
                return 'Style'
        except Exception as e:
            return "Correct, unknown style"
    else: 
        s = row['feedback']
        if s.find('Compilation error') != -1:
            return 'CE'
        if s.find('Runtime error') != -1:
            return 'RE'   
        if s.find('Time limit exceeded') != -1:
            return 'TL'
        if s.find('Perhaps') != -1:
            return 'Check error'      
        if (s.find('Failed test')) or (s.find('Wrong answer')) != -1:
            return 'WA'          
        return 'UND'

In [7]:
df['solution_label'] = df.apply (lambda row: label(row), axis=1)
df

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label
0,5205,2,web,4924,2194,"{""language"": ""python3"", ""code"": ""print(\""Learn...","{""message"": ""Passed. Congratulations! It is yo...",Correct
1,5205,2,PyCharmCore,4757,2736,"{""choices"": null, ""score"": null, ""solution"": n...","{""message"": ""Passed. Congratulations! It is yo...",Correct
2,5205,2,web,4925,3664,"{""language"": ""python3"", ""code"": ""print(\""Learn...","{""message"": ""Passed. Congratulations! It is yo...",Correct
3,5205,2,web,4935,5133,"{""language"": ""python3"", ""code"": ""print(\""Learn...","{""message"": ""Passed. Congratulations! It is yo...",Correct
4,5205,2,web,4773,5726,"{""language"": ""python3"", ""code"": ""print(\""Learn...","{""message"": ""Passed. Congratulations! It is yo...",Correct
...,...,...,...,...,...,...,...,...
4623235,17476,1,web,50035,32258804,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #2 of 4. Wrong answer\n\nThis is ...",WA
4623236,17476,1,web,50035,32261160,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Wrong answer\n\nThis is ...",WA
4623237,17476,2,web,50035,32261307,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct
4623238,17476,2,PyCharmCore,47049,32268590,"{""choices"": null, ""score"": """", ""solution"": nul...","{""message"": """", ""code_style"": {""quality"": {""co...",Style


In [8]:
df.pivot_table(values='id_anon', index = 'client', \
                     columns= 'solution_label', aggfunc= 'count', fill_value=0)

solution_label,CE,Check error,Correct,"Correct, unknown style",RE,Style,TL,WA
client,,,,,,,,
Idea,0,21,871,0,298,390,3,768
IdeaEdu,0,40,1043,0,493,524,7,1043
PyCharmCore,56,1707,67194,1,24568,29660,758,56179
PyCharmEdu,75,2461,92008,1,30799,38196,1059,71626
Python,24,1508,63333,0,19319,27091,459,45950
ide,0,0,21,0,6,7,0,19
idea,2,36,2678,0,460,822,6,1352
internal,0,3,41242,0,1972,36106,70,3336
web,470,46482,1155352,4,1020517,567017,15016,1078349


In [9]:
df.groupby('solution_label')['id_anon'].count()

solution_label
CE                            627
Check error                 52260
Correct                   1472251
Correct, unknown style          6
RE                        1101379
Style                      716434
TL                          17443
WA                        1262840
Name: id_anon, dtype: int64

In [10]:
df_error = df[df['solution_label'] == 'RE']
df_error 

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label
236,5205,1,web,5277,216053,"{""language"": ""python3"", ""code"": ""I love leon.\...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
237,5205,1,web,5277,216098,"{""language"": ""python3"", ""code"": ""I love leo.\n...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
238,5205,1,web,5277,216124,"{""language"": ""python3"", ""code"": ""I love monkey...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
239,5205,1,web,5277,216179,"{""language"": ""python3"", ""code"": ""The monkeys w...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
240,5205,1,web,5277,216218,"{""language"": ""python3"", ""code"": ""The monkey is...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
...,...,...,...,...,...,...,...,...
4623208,17476,1,web,45957,32038448,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE
4623220,17476,1,web,38585,32148433,"{""language"": ""python3"", ""code"": ""lenght = int(...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE
4623227,17476,1,web,49621,32178593,"{""language"": ""python3"", ""code"": ""box_length = ...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE
4623229,17476,1,web,2894,32200330,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE


In [11]:
df_error.head(10)


,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label
236,5205,1,web,5277,216053,"{""language"": ""python3"", ""code"": ""I love leon.\...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
237,5205,1,web,5277,216098,"{""language"": ""python3"", ""code"": ""I love leo.\n...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
238,5205,1,web,5277,216124,"{""language"": ""python3"", ""code"": ""I love monkey...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
239,5205,1,web,5277,216179,"{""language"": ""python3"", ""code"": ""The monkeys w...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
240,5205,1,web,5277,216218,"{""language"": ""python3"", ""code"": ""The monkey is...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
241,5205,1,web,5277,216260,"{""language"": ""python3"", ""code"": ""The monkey ar...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
242,5205,1,web,5277,216520,"{""language"": ""python3"", ""code"": ""The monkey ar...","""Failed. Runtime error\n\nError:\n File \""jai...",RE
273,5205,1,web,5335,247761,"{""language"": ""python3"", ""code"": ""\nPrint(\""Lea...","""Failed. Runtime error\n\nError:\nTraceback (m...",RE
274,5205,1,web,5335,247780,"{""language"": ""python3"", ""code"": ""\nPrint(\""Lea...","""Failed. Runtime error\n\nError:\nTraceback (m...",RE
409,5205,1,web,5561,381230,"{""language"": ""python3"", ""code"": ""\n\n\nPrint(\...","""Failed. Runtime error\n\nError:\nTraceback (m...",RE


In [12]:
df.loc[236,'reply']

'{"language": "python3", "code": "I love leon.\\nThe bird looking good...\\nCamel is hungry.\\n"}'

In [13]:
df.loc[236,'feedback']

'"Failed. Runtime error\\n\\nError:\\n  File \\"jailed_code\\", line 1\\n    I love leon.\\n         ^\\nSyntaxError: invalid syntax\\n"'

In [14]:
import regex as re

def find(string):
    find = re.findall(r'\\n[\w\.]*: .*', string, overlapped=True)
    if 'unrecognized arguments' in string:
        return 'unrecognized arguments' 
    try:
        return find[-1][2:].split('\\n')[0]         
    except:
        if (len(string)>10000):
            return 'Endless output'
        last_line = string.split('\\n')[-2]
        if (len(last_line.split()) == 1):
            return last_line
        return 'error'      
    

find(df.loc[273,'feedback'])

"NameError: name 'Print' is not defined"

In [15]:
df_error['error_message'] = df_error['feedback'].apply(find)
df_error

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,error_message
236,5205,1,web,5277,216053,"{""language"": ""python3"", ""code"": ""I love leon.\...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax
237,5205,1,web,5277,216098,"{""language"": ""python3"", ""code"": ""I love leo.\n...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax
238,5205,1,web,5277,216124,"{""language"": ""python3"", ""code"": ""I love monkey...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax
239,5205,1,web,5277,216179,"{""language"": ""python3"", ""code"": ""The monkeys w...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax
240,5205,1,web,5277,216218,"{""language"": ""python3"", ""code"": ""The monkey is...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax
...,...,...,...,...,...,...,...,...,...
4623208,17476,1,web,45957,32038448,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,NameError: name 'sort' is not defined
4623220,17476,1,web,38585,32148433,"{""language"": ""python3"", ""code"": ""lenght = int(...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,NameError: name 'heght' is not defined
4623227,17476,1,web,49621,32178593,"{""language"": ""python3"", ""code"": ""box_length = ...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,SyntaxError: invalid syntax
4623229,17476,1,web,2894,32200330,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,NameError: name 'false' is not defined


In [16]:
df_error[df_error['error_message'] == 'error']

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,error_message
39698,6267,1,web,24,1869,"{""language"": ""python3"", ""code"": ""import random...","""[ ] Test #1. Runtime error\n[ ] Test #2. Runt...",RE,error
39742,6267,1,web,4202,143564,"{""language"": ""python3"", ""code"": ""import random...","""[ ] Test #1. Runtime error\n[ ] Test #2. Runt...",RE,error
39743,6267,1,web,4202,143838,"{""language"": ""python3"", ""code"": ""import random...","""[ ] Test #1. Runtime error\n[ ] Test #2. Runt...",RE,error
39744,6267,1,web,4202,143885,"{""language"": ""python3"", ""code"": ""import random...","""[ ] Test #1. Runtime error\n[ ] Test #2. Runt...",RE,error
39747,6267,1,web,5025,158478,"{""language"": ""python3"", ""code"": ""import random...","""[ ] Test #1. Runtime error\n[ ] Test #2. Runt...",RE,error
...,...,...,...,...,...,...,...,...,...
4603422,10159,1,web,48938,31903976,"{""language"": ""python3"", ""code"": ""class Area:\n...","""[ ] Test #1. Runtime error\n[ ] Test #2. Runt...",RE,error
4603427,10160,1,web,43474,30998741,"{""language"": ""python3"", ""code"": ""class Time:\n...","""[ ] Test #1. Runtime error\n[ ] Test #2. Runt...",RE,error
4603430,10160,1,web,1865,31163429,"{""language"": ""python3"", ""code"": ""class Time:\n...","""[ ] Test #1. Runtime error\n[ ] Test #2. Runt...",RE,error
4603431,10160,1,web,1865,31163437,"{""language"": ""python3"", ""code"": ""class Time:\n...","""[ ] Test #1. Runtime error\n[ ] Test #2. Runt...",RE,error


In [17]:
df.loc[4603441,'reply']

'{"language": "python3", "code": "class Time:\\n\\n    def __init__(self, hour, minute):\\n        self.hour = hour\\n        self.minute = minute\\n\\n    # use appropriate decorator\\n    @classmethod\\n    def from_string(cls, data):\\n        hour, minute = data.split(\' \')\\n        return cls(hour, minute)\\n"}'

In [18]:
df.loc[4623220,'reply']

'{"language": "python3", "code": "lenght = int(input())\\nwidth = int(input())\\nheight = int(input())\\ndemin = [lenght, width, heght]\\nx = int(input())\\ny = int(input())\\ndemin.sort()\\nif x < y:\\n    if x >= demin[0] and y >= demin[1]:\\n        print(\\"The box can be carried\\")\\nelif x > y:\\n    if y >= demin[0] and x >= demin[1]:\\n        print(\\"The box can be carried\\")\\n    \\nelse:\\n    print(\\"The box cannot be carried\\")\\n"}'

In [19]:
df.loc[4623220,'feedback']

'"Failed test #1 of 4. Runtime error\\n\\nThis is a sample test from the problem statement!\\n\\nTest input:\\n24\\n21\\n11\\n36\\n80\\nCorrect output:\\nThe box can be carried\\n\\nYour code output:\\n\\n\\nError:\\nTraceback (most recent call last):\\n  File \\"jailed_code\\", line 4, in <module>\\n    demin = [lenght, width, heght]\\nNameError: name \'heght\' is not defined\\n"'

In [20]:
df_error['error_type'] = df_error['error_message'].apply(lambda x: x.split(':')[0])
df_error

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,error_message,error_type
236,5205,1,web,5277,216053,"{""language"": ""python3"", ""code"": ""I love leon.\...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
237,5205,1,web,5277,216098,"{""language"": ""python3"", ""code"": ""I love leo.\n...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
238,5205,1,web,5277,216124,"{""language"": ""python3"", ""code"": ""I love monkey...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
239,5205,1,web,5277,216179,"{""language"": ""python3"", ""code"": ""The monkeys w...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
240,5205,1,web,5277,216218,"{""language"": ""python3"", ""code"": ""The monkey is...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
...,...,...,...,...,...,...,...,...,...,...
4623208,17476,1,web,45957,32038448,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,NameError: name 'sort' is not defined,NameError
4623220,17476,1,web,38585,32148433,"{""language"": ""python3"", ""code"": ""lenght = int(...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,NameError: name 'heght' is not defined,NameError
4623227,17476,1,web,49621,32178593,"{""language"": ""python3"", ""code"": ""box_length = ...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,SyntaxError: invalid syntax,SyntaxError
4623229,17476,1,web,2894,32200330,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,NameError: name 'false' is not defined,NameError


In [21]:
df_error.groupby('error_message')['id_anon'].count().nlargest(40)

error_message
SyntaxError: invalid syntax                                                                                      277414
EOFError: EOF when reading a line                                                                                 36872
SyntaxError: EOL while scanning string literal                                                                    28318
TypeError: 'int' object is not iterable                                                                           21929
SyntaxError: unexpected EOF while parsing                                                                         20315
IndentationError: unexpected indent                                                                               14686
IndentationError: expected an indented block                                                                      13647
IndexError: list index out of range                                                                               12518
TypeError: 'str' object is

In [22]:
df_error[df_error['error_type']=="SyntaxError"]

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,error_message,error_type
236,5205,1,web,5277,216053,"{""language"": ""python3"", ""code"": ""I love leon.\...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
237,5205,1,web,5277,216098,"{""language"": ""python3"", ""code"": ""I love leo.\n...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
238,5205,1,web,5277,216124,"{""language"": ""python3"", ""code"": ""I love monkey...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
239,5205,1,web,5277,216179,"{""language"": ""python3"", ""code"": ""The monkeys w...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
240,5205,1,web,5277,216218,"{""language"": ""python3"", ""code"": ""The monkey is...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError
...,...,...,...,...,...,...,...,...,...,...
4623175,17476,1,web,1469,31737440,"{""language"": ""python3"", ""code"": ""box = [int(in...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,SyntaxError: invalid syntax,SyntaxError
4623178,17476,1,web,1469,31737875,"{""language"": ""python3"", ""code"": ""box = [int(in...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,SyntaxError: invalid syntax,SyntaxError
4623207,17476,1,web,45957,32038397,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,SyntaxError: invalid syntax,SyntaxError
4623227,17476,1,web,49621,32178593,"{""language"": ""python3"", ""code"": ""box_length = ...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,SyntaxError: invalid syntax,SyntaxError


In [23]:
def test_failed(string):
    try:
        return int(string.split('#')[1].split(' ')[0])
    except:
        return 0
    
test_failed(df.loc[481980,'feedback'])

1

In [24]:
df_error['number'] = df_error['feedback'].apply(test_failed)
df_error

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,error_message,error_type,number
236,5205,1,web,5277,216053,"{""language"": ""python3"", ""code"": ""I love leon.\...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError,0
237,5205,1,web,5277,216098,"{""language"": ""python3"", ""code"": ""I love leo.\n...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError,0
238,5205,1,web,5277,216124,"{""language"": ""python3"", ""code"": ""I love monkey...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError,0
239,5205,1,web,5277,216179,"{""language"": ""python3"", ""code"": ""The monkeys w...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError,0
240,5205,1,web,5277,216218,"{""language"": ""python3"", ""code"": ""The monkey is...","""Failed. Runtime error\n\nError:\n File \""jai...",RE,SyntaxError: invalid syntax,SyntaxError,0
...,...,...,...,...,...,...,...,...,...,...,...
4623208,17476,1,web,45957,32038448,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,NameError: name 'sort' is not defined,NameError,1
4623220,17476,1,web,38585,32148433,"{""language"": ""python3"", ""code"": ""lenght = int(...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,NameError: name 'heght' is not defined,NameError,1
4623227,17476,1,web,49621,32178593,"{""language"": ""python3"", ""code"": ""box_length = ...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,SyntaxError: invalid syntax,SyntaxError,1
4623229,17476,1,web,2894,32200330,"{""language"": ""python3"", ""code"": ""a = int(input...","""Failed test #1 of 4. Runtime error\n\nThis is...",RE,NameError: name 'false' is not defined,NameError,1


In [25]:
df_error.groupby('number')['step_id'].count()

number
0     420047
1     658062
2      17126
3       2752
4       1222
5        423
6        425
7        604
8        139
9        194
10       175
11        23
12        11
13        63
14         9
15        24
17        18
18         2
26        28
27         8
37        10
43        14
Name: step_id, dtype: int64

In [26]:
df.loc[481980,'feedback'].split('\\n')


['"Failed test #1 of 2. Runtime error',
 '',
 'Error:',
 'usage: 200 [-h]',
 '200: error: unrecognized arguments: --number 200',
 '"']

In [27]:
len(pd.unique(df_error['error_type']))

75

In [28]:
df_error.groupby('error_type')['id_anon'].count().nlargest(40)

error_type
SyntaxError                            382637
TypeError                              256680
NameError                              223669
AttributeError                          65327
EOFError                                36874
ValueError                              35151
IndentationError                        33771
IndexError                              18941
ZeroDivisionError                        6285
KeyError                                 5589
UnboundLocalError                        5136
error                                    4295
FileNotFoundError                        3870
sre_constants.error                      2985
ModuleNotFoundError                      2763
OverflowError                            2722
AssertionError                           1994
requests.exceptions.ConnectionError      1899
RecursionError                           1511
ImportError                              1415
io.UnsupportedOperation                  1296
MemoryError            

In [29]:
df.loc[3858231,'reply']

'{"language": "python3", "code": "b_length = int(input())\\nb_width = int(input())\\nb_height = int(input())\\nd_width = int(input())\\nd_height = int(input())\\n\\ndoor = d_width * d_height\\nbox_size1 = b_length * b_width\\nbox_size2 = b_width * b_height\\nbox_size3 = b_height * b_length\\n\\nif door == box_size1:\\n    print(\\"The box can be carried\\")\\nelif door == box_size2:\\n    print(\\"The box can be carried\\")\\nelif door == box_size3:\\n    print(\\"The box can be carried\\")\\nelse:\\n    print(\\"The box cannot be carried\\")\\n"}'

In [30]:
df.loc[236,'feedback'].split('\\n')[-2]

'SyntaxError: invalid syntax'

In [31]:
df.loc[4622551,'feedback']

'"Failed test #1 of 5. Runtime error\\n\\nThis is a sample test from the problem statement!\\n\\nTest input:\\n567\\nCorrect output:\\n-567\\n\\nYour code output:\\n\\n\\nError:\\nTraceback (most recent call last):\\n  File \\"jailed_code\\", line 2, in <module>\\n    a = int(input)\\nTypeError: int() argument must be a string, a bytes-like object or a number, not \'builtin_function_or_method\'\\n"'

In [32]:
df_tasks = df.pivot_table(values='id_anon', index = 'step_id', \
                     columns= 'solution_label', aggfunc= 'count', fill_value=0)
df_tasks

solution_label,CE,Check error,Correct,"Correct, unknown style",RE,Style,TL,WA
step_id,,,,,,,,
5205,0,5829,20993,0,1537,425,0,25
5209,0,0,16562,0,4667,1770,0,3326
5212,0,0,25493,0,10501,2911,0,18005
5235,0,11069,23950,0,3957,2231,0,3020
5237,0,0,18543,0,3084,1160,2,11559
...,...,...,...,...,...,...,...,...
17433,0,0,2,0,84,96,0,6
17436,0,0,0,0,68,78,0,6
17476,0,0,371,0,415,443,0,868


In [33]:
df_tasks['total'] = df_tasks['CE'] + df_tasks['Check error'] + df_tasks['Correct'] + df_tasks['Correct, unknown style'] + df_tasks['RE'] + df_tasks['Style'] + df_tasks['TL'] + df_tasks['WA']
df_tasks

solution_label,CE,Check error,Correct,"Correct, unknown style",RE,Style,TL,WA,total
step_id,,,,,,,,,
5205,0,5829,20993,0,1537,425,0,25,28809
5209,0,0,16562,0,4667,1770,0,3326,26325
5212,0,0,25493,0,10501,2911,0,18005,56910
5235,0,11069,23950,0,3957,2231,0,3020,44227
5237,0,0,18543,0,3084,1160,2,11559,34348
...,...,...,...,...,...,...,...,...,...
17433,0,0,2,0,84,96,0,6,188
17436,0,0,0,0,68,78,0,6,152
17476,0,0,371,0,415,443,0,868,2097


In [34]:
df_tasks['rate'] = (df_tasks['Correct'] + df_tasks['Correct, unknown style'] + df_tasks['Style'])/df_tasks['total']
df_tasks

solution_label,CE,Check error,Correct,"Correct, unknown style",RE,Style,TL,WA,total,rate
step_id,,,,,,,,,,
5205,0,5829,20993,0,1537,425,0,25,28809,0.743448
5209,0,0,16562,0,4667,1770,0,3326,26325,0.696372
5212,0,0,25493,0,10501,2911,0,18005,56910,0.499104
5235,0,11069,23950,0,3957,2231,0,3020,44227,0.591969
5237,0,0,18543,0,3084,1160,2,11559,34348,0.573629
...,...,...,...,...,...,...,...,...,...,...
17433,0,0,2,0,84,96,0,6,188,0.521277
17436,0,0,0,0,68,78,0,6,152,0.513158
17476,0,0,371,0,415,443,0,868,2097,0.388174


In [35]:
df_tasks[(df_tasks['rate']<0.5) & (df_tasks['total']>1000)]

solution_label,CE,Check error,Correct,"Correct, unknown style",RE,Style,TL,WA,total,rate
step_id,,,,,,,,,,
5212,0,0,25493,0,10501,2911,0,18005,56910,0.499104
5850,0,450,769,0,857,481,7,148,2712,0.460914
5889,0,0,12215,0,21312,736,0,3996,38259,0.338509
5907,0,0,4978,0,3859,2060,288,7487,18672,0.376928
5919,0,0,6696,0,8248,5695,2,4209,24850,0.498632
...,...,...,...,...,...,...,...,...,...,...
15944,0,0,1612,0,977,497,1,1430,4517,0.466903
15945,64,0,601,0,414,100,0,355,1534,0.456975
15965,0,0,28,0,251,238,0,486,1003,0.265204


In [36]:
df_test = df[(df['step_id']==15944) & (df['solution_label']!='WA') & (df['solution_label']!='Style')]
df_test

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label
3122670,15944,2,web,32934,18602893,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct
3122673,15944,1,web,26245,18605392,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE
3122677,15944,2,web,27006,18605599,"{""language"": ""python3"", ""code"": ""num = int(inp...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct
3122680,15944,1,web,26245,18605786,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE
3122683,15944,2,web,33310,18608888,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct
...,...,...,...,...,...,...,...,...
4622563,15944,2,web,47462,32243973,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct
4622564,15944,1,web,50966,32258083,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE
4622567,15944,1,web,50966,32258180,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE
4622569,15944,2,web,50966,32258309,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct


In [37]:
df.loc[4623239,'reply']

'{"choices": null, "score": "", "solution": null, "language": "python3", "code": "box_a = int(input())\\nbox_b = int(input())\\nbox_c = int(input())\\ndoor_x = int(input())\\ndoor_y = int(input())\\n\\nif door_y + door_y < (box_a + box_b or box_a + box_c or box_b + box_c):\\n    print(\\"The box can be carried\\")\\nelse:\\n    print(\\"The box cannot be carried\\")\\n", "text": null, "number": null, "edu_task": null, "version": 13, "feedback": null}'

In [38]:
def find_line(string):
    text = re.findall(r'line \d+', string)
    if (text):
        return int(text[0].split()[-1])
    return 0

find_line(df.loc[3122670,'feedback'])

0

In [39]:
df_test['line'] = df_test['feedback'].apply(find_line)
df_test

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,line
3122670,15944,2,web,32934,18602893,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0
3122673,15944,1,web,26245,18605392,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,5
3122677,15944,2,web,27006,18605599,"{""language"": ""python3"", ""code"": ""num = int(inp...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0
3122680,15944,1,web,26245,18605786,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,7
3122683,15944,2,web,33310,18608888,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0
...,...,...,...,...,...,...,...,...,...
4622563,15944,2,web,47462,32243973,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0
4622564,15944,1,web,50966,32258083,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3
4622567,15944,1,web,50966,32258180,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3
4622569,15944,2,web,50966,32258309,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0


In [8]:
def find_code(string):
    try:
        return json.loads(string, strict=False)['code'] 
    except BaseException:
        return ""

In [11]:
df_test['code'] = df_test['reply'].apply(find_code)
df_test

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,code
3122668,15944,1,web,32934,18602813,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\n\r...
3122669,15944,1,web,32934,18602843,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\n\r...
3122670,15944,2,web,32934,18602893,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\na = int(input())\r\n\r...
3122671,15944,1,web,30310,18604036,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Wrong answer\n\nThis is ...",WA,"# put your code here\nA = int(input(""""))\nprin..."
3122672,15944,2,web,30310,18604202,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Style,"# put your code here\nA = int(input(""""))\nprin..."
...,...,...,...,...,...,...,...,...,...
4622567,15944,1,web,50966,32258180,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,# put your code here\r\na = input()\r\nif a>0:...
4622568,15944,1,web,50966,32258224,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\nif...
4622569,15944,2,web,50966,32258309,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\nprint(-(int(input())))\n
4622570,15944,2,web,44573,32275474,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,a = int(input())\r\nprint(a * -1)\n


In [42]:
df_test.groupby('solution_label')['id_anon'].count()

solution_label
Correct    1612
RE          977
TL            1
Name: id_anon, dtype: int64

In [43]:
df_positive = df_test[df_test['status'] == 2]
df_positive

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,line,code
3122670,15944,2,web,32934,18602893,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0,# put your code here\r\na = int(input())\r\n\r...
3122677,15944,2,web,27006,18605599,"{""language"": ""python3"", ""code"": ""num = int(inp...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0,num = int(input())\nprint(-num)\n
3122683,15944,2,web,33310,18608888,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0,# put your code here\nx = int(input())\nprint(...
3122686,15944,2,web,1971,18610072,"{""language"": ""python3"", ""code"": ""num = int(inp...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0,num = int(input())\nprint(num * -1)\n
3122687,15944,2,web,34615,18610261,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0,# put your code here\nnumber = int(input())\np...
...,...,...,...,...,...,...,...,...,...,...
4622560,15944,2,PyCharmCore,468,32193348,"{""choices"": null, ""score"": """", ""solution"": nul...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0,print(-int(input()))\n
4622561,15944,2,web,50311,32222897,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0,# put your code here\r\nnum = - int(input())\r...
4622563,15944,2,web,47462,32243973,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0,a = int(input())\nprint(-a)\n
4622569,15944,2,web,50966,32258309,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,0,# put your code here\r\nprint(-(int(input())))\n


In [44]:
df_re = df_test[df_test['solution_label'] == "RE"]
df_re

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,line,code
3122673,15944,1,web,26245,18605392,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,5,# put your code here\ninteg = int(input())\nif...
3122680,15944,1,web,26245,18605786,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,7,# put your code here\ninteg = int(input())\nif...
3122685,15944,1,web,1971,18609996,"{""language"": ""python3"", ""code"": ""usr = input()...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,2,usr = input()\nreturn(usr * -1)\n
3122690,15944,1,web,34546,18617888,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3,# put your code here\nif x < 0:\n print x\n...
3122691,15944,1,web,34546,18617910,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,2,# put your code here\nif x < 0:\n print(x)\...
...,...,...,...,...,...,...,...,...,...,...
4622552,15944,1,web,50966,32175218,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,2,# put your code here\r\na = int(input)\r\nif a...
4622553,15944,1,web,50966,32175257,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3,# put your code here\r\na = str(input)\r\nif a...
4622554,15944,1,web,50966,32175315,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3,# put your code here\r\na = str(input)\r\nif i...
4622564,15944,1,web,50966,32258083,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3,# put your code here\r\na = input()\r\nprint(-...


In [45]:
df.loc[3122673,'feedback'].split('\\n')


['"Failed test #1 of 5. Runtime error',
 '',
 'This is a sample test from the problem statement!',
 '',
 'Test input:',
 '567',
 'Correct output:',
 '-567',
 '',
 'Your code output:',
 '',
 '',
 'Error:',
 '  File \\"jailed_code\\", line 5',
 '    print integ',
 '              ^',
 "SyntaxError: Missing parentheses in call to 'print'",
 '"']

In [46]:
df_neural = pd.DataFrame({'code' : df_test['code'], 'line' : df_test['line']})
df_neural

,code,line
3122670,# put your code here\r\na = int(input())\r\n\r...,0
3122673,# put your code here\ninteg = int(input())\nif...,5
3122677,num = int(input())\nprint(-num)\n,0
3122680,# put your code here\ninteg = int(input())\nif...,7
3122683,# put your code here\nx = int(input())\nprint(...,0
...,...,...
4622563,a = int(input())\nprint(-a)\n,0
4622564,# put your code here\r\na = input()\r\nprint(-...,3
4622567,# put your code here\r\na = input()\r\nif a>0:...,3
4622569,# put your code here\r\nprint(-(int(input())))\n,0


In [47]:
df_neural.groupby('line')['line'].count()

line
0     1613
1       80
2      316
3      281
4      129
5       84
6       48
7       14
8       15
9        6
10       1
14       1
15       1
25       1
Name: line, dtype: int64

In [14]:
from nltk.tokenize import word_tokenize

def tokenize(string):
    text = re.sub('#.*', '', string) 
    return word_tokenize(text)

In [50]:
df_neural['code_clean'] = df_neural['code'].apply(tokenize)
df_neural

,code,line,code_clean
3122670,# put your code here\r\na = int(input())\r\n\r...,0,"[a, =, int, (, input, (, ), ), print, (, a, *,..."
3122673,# put your code here\ninteg = int(input())\nif...,5,"[integ, =, int, (, input, (, ), ), if, integ, ..."
3122677,num = int(input())\nprint(-num)\n,0,"[num, =, int, (, input, (, ), ), print, (, -nu..."
3122680,# put your code here\ninteg = int(input())\nif...,7,"[integ, =, int, (, input, (, ), ), if, integ, ..."
3122683,# put your code here\nx = int(input())\nprint(...,0,"[x, =, int, (, input, (, ), ), print, (, -x, )]"
...,...,...,...
4622563,a = int(input())\nprint(-a)\n,0,"[a, =, int, (, input, (, ), ), print, (, -a, )]"
4622564,# put your code here\r\na = input()\r\nprint(-...,3,"[a, =, input, (, ), print, (, -a, )]"
4622567,# put your code here\r\na = input()\r\nif a>0:...,3,"[a, =, input, (, ), if, a, >, 0, :, print, (, ..."
4622569,# put your code here\r\nprint(-(int(input())))\n,0,"[print, (, -, (, int, (, input, (, ), ), ), )]"


In [15]:
language_construction = ['False', 'True', 'await', 'else','elif', 'if', 'import', 'pass', 'None', 'break', 'except', 'in', 'raise', 'class', 
                         'finally', 'is', 'return', 'and', 'continue', 'for', 'lambda', 'try', 'as', 'def', 'from', 'nonlocal', 
                         'while', 'assert', 'del', 'global', 'not', 'with', 'async', 'or', 'yield']

In [16]:
dict_language = {value: index for index, value in enumerate(language_construction)}

In [17]:
dict_token = {value: index for index, value in enumerate(language_construction)}
dict_token

{'False': 0,
 'True': 1,
 'await': 2,
 'else': 3,
 'elif': 4,
 'if': 5,
 'import': 6,
 'pass': 7,
 'None': 8,
 'break': 9,
 'except': 10,
 'in': 11,
 'raise': 12,
 'class': 13,
 'finally': 14,
 'is': 15,
 'return': 16,
 'and': 17,
 'continue': 18,
 'for': 19,
 'lambda': 20,
 'try': 21,
 'as': 22,
 'def': 23,
 'from': 24,
 'nonlocal': 25,
 'while': 26,
 'assert': 27,
 'del': 28,
 'global': 29,
 'not': 30,
 'with': 31,
 'async': 32,
 'or': 33,
 'yield': 34}

In [18]:
index_dict = len(language_construction)
index_dict

35

In [19]:
dict_token['False']

0

In [20]:
symbols = ['!', '?','_', '“', '#', '$', '%', '&', '‘', '(',')', '*', '+', '-', '.', '/', ':', ';', '<', '=', '>', '@', '\\', '[', ']', '{', '}', '|', '~']

In [21]:
dict_language.update({value: index+index_dict for index, value in enumerate(symbols)})
dict_language

{'False': 0,
 'True': 1,
 'await': 2,
 'else': 3,
 'elif': 4,
 'if': 5,
 'import': 6,
 'pass': 7,
 'None': 8,
 'break': 9,
 'except': 10,
 'in': 11,
 'raise': 12,
 'class': 13,
 'finally': 14,
 'is': 15,
 'return': 16,
 'and': 17,
 'continue': 18,
 'for': 19,
 'lambda': 20,
 'try': 21,
 'as': 22,
 'def': 23,
 'from': 24,
 'nonlocal': 25,
 'while': 26,
 'assert': 27,
 'del': 28,
 'global': 29,
 'not': 30,
 'with': 31,
 'async': 32,
 'or': 33,
 'yield': 34,
 '!': 35,
 '?': 36,
 '_': 37,
 '“': 38,
 '#': 39,
 '$': 40,
 '%': 41,
 '&': 42,
 '‘': 43,
 '(': 44,
 ')': 45,
 '*': 46,
 '+': 47,
 '-': 48,
 '.': 49,
 '/': 50,
 ':': 51,
 ';': 52,
 '<': 53,
 '=': 54,
 '>': 55,
 '@': 56,
 '\\': 57,
 '[': 58,
 ']': 59,
 '{': 60,
 '}': 61,
 '|': 62,
 '~': 63}

In [22]:
dict_token.update({value: index+index_dict for index, value in enumerate(symbols)})
dict_token

{'False': 0,
 'True': 1,
 'await': 2,
 'else': 3,
 'elif': 4,
 'if': 5,
 'import': 6,
 'pass': 7,
 'None': 8,
 'break': 9,
 'except': 10,
 'in': 11,
 'raise': 12,
 'class': 13,
 'finally': 14,
 'is': 15,
 'return': 16,
 'and': 17,
 'continue': 18,
 'for': 19,
 'lambda': 20,
 'try': 21,
 'as': 22,
 'def': 23,
 'from': 24,
 'nonlocal': 25,
 'while': 26,
 'assert': 27,
 'del': 28,
 'global': 29,
 'not': 30,
 'with': 31,
 'async': 32,
 'or': 33,
 'yield': 34,
 '!': 35,
 '?': 36,
 '_': 37,
 '“': 38,
 '#': 39,
 '$': 40,
 '%': 41,
 '&': 42,
 '‘': 43,
 '(': 44,
 ')': 45,
 '*': 46,
 '+': 47,
 '-': 48,
 '.': 49,
 '/': 50,
 ':': 51,
 ';': 52,
 '<': 53,
 '=': 54,
 '>': 55,
 '@': 56,
 '\\': 57,
 '[': 58,
 ']': 59,
 '{': 60,
 '}': 61,
 '|': 62,
 '~': 63}

In [23]:
def vectorize(token_list):
    result_list = []
    for w in token_list:
        if w in dict_token:
            result_list.append(dict_token[w])
        else:
            dict_token[w] = len(dict_token)
            result_list.append(dict_token[w])
    return result_list

In [58]:
dict_token

{'False': 0,
 'True': 1,
 'await': 2,
 'else': 3,
 'elif': 4,
 'if': 5,
 'import': 6,
 'pass': 7,
 'None': 8,
 'break': 9,
 'except': 10,
 'in': 11,
 'raise': 12,
 'class': 13,
 'finally': 14,
 'is': 15,
 'return': 16,
 'and': 17,
 'continue': 18,
 'for': 19,
 'lambda': 20,
 'try': 21,
 'as': 22,
 'def': 23,
 'from': 24,
 'nonlocal': 25,
 'while': 26,
 'assert': 27,
 'del': 28,
 'global': 29,
 'not': 30,
 'with': 31,
 'async': 32,
 'or': 33,
 'yield': 34,
 '!': 35,
 '?': 36,
 '_': 37,
 '“': 38,
 '#': 39,
 '$': 40,
 '%': 41,
 '&': 42,
 '‘': 43,
 '(': 44,
 ')': 45,
 '*': 46,
 '+': 47,
 '-': 48,
 '.': 49,
 '/': 50,
 ':': 51,
 ';': 52,
 '<': 53,
 '=': 54,
 '>': 55,
 '@': 56,
 '\\': 57,
 '[': 58,
 ']': 59,
 '{': 60,
 '}': 61,
 '|': 62,
 '~': 63}

In [59]:
df_neural['vector'] = df_neural['code_clean'].apply(vectorize)
df_neural

,code,line,code_clean,vector
3122670,# put your code here\r\na = int(input())\r\n\r...,0,"[a, =, int, (, input, (, ), ), print, (, a, *,...","[64, 54, 65, 44, 66, 44, 45, 45, 67, 44, 64, 4..."
3122673,# put your code here\ninteg = int(input())\nif...,5,"[integ, =, int, (, input, (, ), ), if, integ, ...","[69, 54, 65, 44, 66, 44, 45, 45, 5, 69, 53, 70..."
3122677,num = int(input())\nprint(-num)\n,0,"[num, =, int, (, input, (, ), ), print, (, -nu...","[72, 54, 65, 44, 66, 44, 45, 45, 67, 44, 73, 45]"
3122680,# put your code here\ninteg = int(input())\nif...,7,"[integ, =, int, (, input, (, ), ), if, integ, ...","[69, 54, 65, 44, 66, 44, 45, 45, 5, 69, 55, 70..."
3122683,# put your code here\nx = int(input())\nprint(...,0,"[x, =, int, (, input, (, ), ), print, (, -x, )]","[74, 54, 65, 44, 66, 44, 45, 45, 67, 44, 75, 45]"
...,...,...,...,...
4622563,a = int(input())\nprint(-a)\n,0,"[a, =, int, (, input, (, ), ), print, (, -a, )]","[64, 54, 65, 44, 66, 44, 45, 45, 67, 44, 83, 45]"
4622564,# put your code here\r\na = input()\r\nprint(-...,3,"[a, =, input, (, ), print, (, -a, )]","[64, 54, 66, 44, 45, 67, 44, 83, 45]"
4622567,# put your code here\r\na = input()\r\nif a>0:...,3,"[a, =, input, (, ), if, a, >, 0, :, print, (, ...","[64, 54, 66, 44, 45, 5, 64, 55, 70, 51, 67, 44..."
4622569,# put your code here\r\nprint(-(int(input())))\n,0,"[print, (, -, (, int, (, input, (, ), ), ), )]","[67, 44, 48, 44, 65, 44, 66, 44, 45, 45, 45, 45]"


In [60]:
df_neural['vector_size'] = df_neural['vector'].apply(len)
df_neural

,code,line,code_clean,vector,vector_size
3122670,# put your code here\r\na = int(input())\r\n\r...,0,"[a, =, int, (, input, (, ), ), print, (, a, *,...","[64, 54, 65, 44, 66, 44, 45, 45, 67, 44, 64, 4...",14
3122673,# put your code here\ninteg = int(input())\nif...,5,"[integ, =, int, (, input, (, ), ), if, integ, ...","[69, 54, 65, 44, 66, 44, 45, 45, 5, 69, 53, 70...",48
3122677,num = int(input())\nprint(-num)\n,0,"[num, =, int, (, input, (, ), ), print, (, -nu...","[72, 54, 65, 44, 66, 44, 45, 45, 67, 44, 73, 45]",12
3122680,# put your code here\ninteg = int(input())\nif...,7,"[integ, =, int, (, input, (, ), ), if, integ, ...","[69, 54, 65, 44, 66, 44, 45, 45, 5, 69, 55, 70...",32
3122683,# put your code here\nx = int(input())\nprint(...,0,"[x, =, int, (, input, (, ), ), print, (, -x, )]","[74, 54, 65, 44, 66, 44, 45, 45, 67, 44, 75, 45]",12
...,...,...,...,...,...
4622563,a = int(input())\nprint(-a)\n,0,"[a, =, int, (, input, (, ), ), print, (, -a, )]","[64, 54, 65, 44, 66, 44, 45, 45, 67, 44, 83, 45]",12
4622564,# put your code here\r\na = input()\r\nprint(-...,3,"[a, =, input, (, ), print, (, -a, )]","[64, 54, 66, 44, 45, 67, 44, 83, 45]",9
4622567,# put your code here\r\na = input()\r\nif a>0:...,3,"[a, =, input, (, ), if, a, >, 0, :, print, (, ...","[64, 54, 66, 44, 45, 5, 64, 55, 70, 51, 67, 44...",34
4622569,# put your code here\r\nprint(-(int(input())))\n,0,"[print, (, -, (, int, (, input, (, ), ), ), )]","[67, 44, 48, 44, 65, 44, 66, 44, 45, 45, 45, 45]",12


In [61]:
df_neural['vector_size'].min()

6

In [62]:
df_neural['vector'].values

array([list([64, 54, 65, 44, 66, 44, 45, 45, 67, 44, 64, 46, 68, 45]),
       list([69, 54, 65, 44, 66, 44, 45, 45, 5, 69, 53, 70, 51, 69, 54, 69, 47, 44, 71, 46, 69, 45, 67, 69, 4, 69, 55, 70, 51, 69, 54, 69, 48, 44, 71, 46, 69, 45, 67, 44, 69, 45, 3, 51, 67, 44, 69, 45]),
       list([72, 54, 65, 44, 66, 44, 45, 45, 67, 44, 73, 45]), ...,
       list([64, 54, 66, 44, 45, 5, 64, 55, 70, 51, 67, 44, 103, 48, 81, 47, 84, 44, 64, 45, 45, 3, 51, 67, 44, 103, 47, 81, 47, 84, 44, 64, 45, 45]),
       list([67, 44, 48, 44, 65, 44, 66, 44, 45, 45, 45, 45]),
       list([64, 54, 65, 44, 66, 44, 45, 45, 67, 44, 64, 46, 68, 45])],
      dtype=object)

In [63]:
max_features = len(dict_token)
maxlen = 300 

In [64]:
import torch
import keras

X = keras.preprocessing.sequence.pad_sequences(df_neural['vector'], maxlen=maxlen)
y = df_neural['line'].values

In [65]:
X.shape


(2590, 300)

In [88]:
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Activation, Dense
import tensorflow as tf


tf.keras.backend.clear_session()
model = Sequential()
model.add(LSTM(60, activation='relu', input_shape=(maxlen, 1), bias_regularizer = regularizers.L1(0.01)))
model.add(Dense(10, activation='relu'))
#model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1))

model.summary()

model.compile(optimizer='adam', loss='mse', metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60)                14880     
                                                                 
 dense (Dense)               (None, 10)                610       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 15,501
Trainable params: 15,501
Non-trainable params: 0
_________________________________________________________________


In [89]:
history = model.fit(X, y, batch_size=128, epochs=1)

21/21 [==============================] - 7s 289ms/step - loss: 5804.4746 - accuracy: 0.5305


In [105]:
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense
import tensorflow as tf


tf.keras.backend.clear_session()
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(maxlen, 1), bias_regularizer = regularizers.L1(0.01)))
#model.add(Dense(10, activation='relu'))
#model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1))

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 50)                2600      
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 2,651
Trainable params: 2,651
Non-trainable params: 0
_________________________________________________________________


In [106]:
history = model.fit(X, y, batch_size=128, epochs=1)

21/21 [==============================] - 2s 72ms/step - loss: 1.4337e-07 - accuracy: 0.6228


In [ ]:
tf.keras.backend.clear_session()
model = Sequential()
model.add(LSTM(30, activation='relu', input_shape=(maxlen, 1)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.summary()

model.compile(optimizer='adam', loss='mse', metrics=["accuracy"])

In [ ]:
model.save('finalized_model')

In [ ]:
from tensorflow import keras
model = keras.models.load_model('finalized_model')


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_neural, test_size=0.1)


In [ ]:
X_test = keras.preprocessing.sequence.pad_sequences(test['vector'], maxlen=maxlen)
y_test = test['line'].values

In [ ]:
model.evaluate(X_test,y_test)

In [107]:
for e in zip(model.layers[0].trainable_weights, model.layers[0].get_weights()):
    print('Param %s:\n%s' % (e[0],e[1]))

Param <tf.Variable 'simple_rnn/simple_rnn_cell/kernel:0' shape=(1, 50) dtype=float32, numpy=
array([[ 0.3395762 , -0.00735486, -0.11848442,  0.04346287,  0.3130001 ,
        -0.0785754 ,  0.28733286, -0.19289395, -0.16711141, -0.1414028 ,
        -0.15905394,  0.32188287, -0.28178528, -0.07597154, -0.00660259,
         0.05579203,  0.13864815,  0.29055396,  0.03138924, -0.0974227 ,
        -0.020942  ,  0.23536184, -0.2831607 ,  0.11140826, -0.30780923,
        -0.13750057, -0.02949122,  0.15296042,  0.09198993,  0.05864221,
        -0.00589138, -0.0062817 , -0.25389755,  0.2690573 ,  0.2941008 ,
         0.00350347,  0.30094203,  0.2996734 , -0.18135294, -0.00684693,
        -0.03476802,  0.336315  ,  0.14674634,  0.33704743, -0.18979877,
         0.04925734,  0.28870836,  0.15947196,  0.03779089,  0.31955573]],
      dtype=float32)>:
[[ 0.3395762  -0.00735486 -0.11848442  0.04346287  0.3130001  -0.0785754
   0.28733286 -0.19289395 -0.16711141 -0.1414028  -0.15905394  0.32188287
  -0.

In [108]:
model.layers[0].weights

[<tf.Variable 'simple_rnn/simple_rnn_cell/kernel:0' shape=(1, 50) dtype=float32, numpy=
 array([[ 0.3395762 , -0.00735486, -0.11848442,  0.04346287,  0.3130001 ,
         -0.0785754 ,  0.28733286, -0.19289395, -0.16711141, -0.1414028 ,
         -0.15905394,  0.32188287, -0.28178528, -0.07597154, -0.00660259,
          0.05579203,  0.13864815,  0.29055396,  0.03138924, -0.0974227 ,
         -0.020942  ,  0.23536184, -0.2831607 ,  0.11140826, -0.30780923,
         -0.13750057, -0.02949122,  0.15296042,  0.09198993,  0.05864221,
         -0.00589138, -0.0062817 , -0.25389755,  0.2690573 ,  0.2941008 ,
          0.00350347,  0.30094203,  0.2996734 , -0.18135294, -0.00684693,
         -0.03476802,  0.336315  ,  0.14674634,  0.33704743, -0.18979877,
          0.04925734,  0.28870836,  0.15947196,  0.03779089,  0.31955573]],
       dtype=float32)>,
 <tf.Variable 'simple_rnn/simple_rnn_cell/recurrent_kernel:0' shape=(50, 50) dtype=float32, numpy=
 array([[ 0.12516797, -0.18377697,  0.18016587,

In [ ]:
y_res = model.predict(X_test)

In [ ]:
y_res

In [ ]:
layer_weights = model.layers[0].get_weights()
layer_weights

In [ ]:
from see_rnn import get_layer_gradients, features_0D, features_1D, features_2D

grads = get_layer_gradients(model, 1, x, y)  # return_sequences=True,  layer index 1
grads = get_layer_gradients(model, 2, x, y)  # return_sequences=False, layer index 2
outs  = get_outputs(model, 1, x)       # return_sequences=True,  layer index 1

In [ ]:
tf.keras.backend.clear_session()
model1 = Sequential()
model1.add(LSTM(30, activation='relu', input_shape=(maxlen, 1)))
model1.add(Dense(1))

model1.summary()

model1.compile(optimizer='adam', loss='mse', metrics=["accuracy"])

In [ ]:
history = model1.fit(X, y, batch_size=32, epochs=1)

In [ ]:
#64 32 32 40%
from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 64)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(32))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [109]:
df_re['error_message'] = df_re['feedback'].apply(find)
df_re

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,line,code,error_message
3122673,15944,1,web,26245,18605392,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,5,# put your code here\ninteg = int(input())\nif...,SyntaxError: Missing parentheses in call to 'p...
3122680,15944,1,web,26245,18605786,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,7,# put your code here\ninteg = int(input())\nif...,SyntaxError: can't assign to operator
3122685,15944,1,web,1971,18609996,"{""language"": ""python3"", ""code"": ""usr = input()...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,2,usr = input()\nreturn(usr * -1)\n,SyntaxError: 'return' outside function
3122690,15944,1,web,34546,18617888,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3,# put your code here\nif x < 0:\n print x\n...,SyntaxError: Missing parentheses in call to 'p...
3122691,15944,1,web,34546,18617910,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,2,# put your code here\nif x < 0:\n print(x)\...,NameError: name 'x' is not defined
...,...,...,...,...,...,...,...,...,...,...,...
4622552,15944,1,web,50966,32175218,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,2,# put your code here\r\na = int(input)\r\nif a...,"TypeError: int() argument must be a string, a ..."
4622553,15944,1,web,50966,32175257,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3,# put your code here\r\na = str(input)\r\nif a...,TypeError: '>' not supported between instances...
4622554,15944,1,web,50966,32175315,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3,# put your code here\r\na = str(input)\r\nif i...,ValueError: invalid literal for int() with bas...
4622564,15944,1,web,50966,32258083,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,3,# put your code here\r\na = input()\r\nprint(-...,TypeError: bad operand type for unary -: 'str'


In [110]:
df_re.groupby('error_message')['id_anon'].count().nlargest(40)

error_message
SyntaxError: invalid syntax                                                                                      305
TypeError: must be str, not int                                                                                  103
TypeError: int() argument must be a string, a bytes-like object or a number, not 'builtin_function_or_method'     89
SyntaxError: unexpected EOF while parsing                                                                         86
TypeError: bad operand type for unary -: 'str'                                                                    48
TypeError: '>' not supported between instances of 'str' and 'int'                                                 32
TypeError: bad operand type for abs(): 'str'                                                                      23
IndentationError: unexpected indent                                                                               19
TypeError: '<' not supported between instances of 

In [ ]:
def clean(s):
    s1 = re.sub("\'[^']*\'","",s)
    s1 = re.sub("\\([^']*\\)","",s1)
    s1 = re.sub("\\b[0-9][0-9.,-_]*\\b", '', s1)
    s1 = re.sub (r' +', ' ',s1)
    find = re.compile(r"^[^.\\:]*")
    return s1

In [ ]:
df_re.groupby('error_message_clean')['id_anon'].count().nlargest(40)

In [10]:
df_test = df[df['step_id']==15944]
df_test

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label
3122668,15944,1,web,32934,18602813,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA
3122669,15944,1,web,32934,18602843,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA
3122670,15944,2,web,32934,18602893,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct
3122671,15944,1,web,30310,18604036,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Wrong answer\n\nThis is ...",WA
3122672,15944,2,web,30310,18604202,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Style
...,...,...,...,...,...,...,...,...
4622567,15944,1,web,50966,32258180,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE
4622568,15944,1,web,50966,32258224,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA
4622569,15944,2,web,50966,32258309,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct
4622570,15944,2,web,44573,32275474,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct


In [12]:
df_test['code'] = df_test['reply'].apply(find_code)
df_test

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,code
3122668,15944,1,web,32934,18602813,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\n\r...
3122669,15944,1,web,32934,18602843,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\n\r...
3122670,15944,2,web,32934,18602893,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\na = int(input())\r\n\r...
3122671,15944,1,web,30310,18604036,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Wrong answer\n\nThis is ...",WA,"# put your code here\nA = int(input(""""))\nprin..."
3122672,15944,2,web,30310,18604202,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Style,"# put your code here\nA = int(input(""""))\nprin..."
...,...,...,...,...,...,...,...,...,...
4622567,15944,1,web,50966,32258180,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,# put your code here\r\na = input()\r\nif a>0:...
4622568,15944,1,web,50966,32258224,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\nif...
4622569,15944,2,web,50966,32258309,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\nprint(-(int(input())))\n
4622570,15944,2,web,44573,32275474,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,a = int(input())\r\nprint(a * -1)\n


In [24]:
df_test['code_clean'] = df_test['code'].apply(tokenize)
df_test

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,code,code_clean
3122668,15944,1,web,32934,18602813,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\n\r...,"[a, =, int, (, input, (, ), ), if, a, <, 0, :,..."
3122669,15944,1,web,32934,18602843,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\n\r...,"[a, =, int, (, input, (, ), ), if, a, <, 0, :,..."
3122670,15944,2,web,32934,18602893,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\na = int(input())\r\n\r...,"[a, =, int, (, input, (, ), ), print, (, a, *,..."
3122671,15944,1,web,30310,18604036,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Wrong answer\n\nThis is ...",WA,"# put your code here\nA = int(input(""""))\nprin...","[A, =, int, (, input, (, ``, '', ), ), print, ..."
3122672,15944,2,web,30310,18604202,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Style,"# put your code here\nA = int(input(""""))\nprin...","[A, =, int, (, input, (, ``, '', ), ), print, ..."
...,...,...,...,...,...,...,...,...,...,...
4622567,15944,1,web,50966,32258180,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,# put your code here\r\na = input()\r\nif a>0:...,"[a, =, input, (, ), if, a, >, 0, :, print, (, ..."
4622568,15944,1,web,50966,32258224,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\nif...,"[a, =, int, (, input, (, ), ), if, a, >, 0, :,..."
4622569,15944,2,web,50966,32258309,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\nprint(-(int(input())))\n,"[print, (, -, (, int, (, input, (, ), ), ), )]"
4622570,15944,2,web,44573,32275474,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,a = int(input())\r\nprint(a * -1)\n,"[a, =, int, (, input, (, ), ), print, (, a, *,..."


In [25]:
df_test['vector'] = df_test['code_clean'].apply(vectorize)
df_test

,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,code,code_clean,vector
3122668,15944,1,web,32934,18602813,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\n\r...,"[a, =, int, (, input, (, ), ), if, a, <, 0, :,...","[64, 54, 65, 44, 66, 44, 45, 45, 5, 64, 53, 67..."
3122669,15944,1,web,32934,18602843,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\n\r...,"[a, =, int, (, input, (, ), ), if, a, <, 0, :,...","[64, 54, 65, 44, 66, 44, 45, 45, 5, 64, 53, 67..."
3122670,15944,2,web,32934,18602893,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\na = int(input())\r\n\r...,"[a, =, int, (, input, (, ), ), print, (, a, *,...","[64, 54, 65, 44, 66, 44, 45, 45, 68, 44, 64, 4..."
3122671,15944,1,web,30310,18604036,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Wrong answer\n\nThis is ...",WA,"# put your code here\nA = int(input(""""))\nprin...","[A, =, int, (, input, (, ``, '', ), ), print, ...","[70, 54, 65, 44, 66, 44, 71, 72, 45, 45, 68, 4..."
3122672,15944,2,web,30310,18604202,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Style,"# put your code here\nA = int(input(""""))\nprin...","[A, =, int, (, input, (, ``, '', ), ), print, ...","[70, 54, 65, 44, 66, 44, 71, 72, 45, 45, 68, 4..."
...,...,...,...,...,...,...,...,...,...,...,...
4622567,15944,1,web,50966,32258180,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #1 of 5. Runtime error\n\nThis is...",RE,# put your code here\r\na = input()\r\nif a>0:...,"[a, =, input, (, ), if, a, >, 0, :, print, (, ...","[64, 54, 66, 44, 45, 5, 64, 55, 67, 51, 68, 44..."
4622568,15944,1,web,50966,32258224,"{""language"": ""python3"", ""code"": ""# put your co...","""Failed test #2 of 5. Wrong answer\n""",WA,# put your code here\r\na = int(input())\r\nif...,"[a, =, int, (, input, (, ), ), if, a, >, 0, :,...","[64, 54, 65, 44, 66, 44, 45, 45, 5, 64, 55, 67..."
4622569,15944,2,web,50966,32258309,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\nprint(-(int(input())))\n,"[print, (, -, (, int, (, input, (, ), ), ), )]","[68, 44, 48, 44, 65, 44, 66, 44, 45, 45, 45, 45]"
4622570,15944,2,web,44573,32275474,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,a = int(input())\r\nprint(a * -1)\n,"[a, =, int, (, input, (, ), ), print, (, a, *,...","[64, 54, 65, 44, 66, 44, 45, 45, 68, 44, 64, 4..."


In [49]:
users = pd.unique(df_test['uid_anon'])
users

array([32934, 30310, 26245, ..., 47462, 44573, 51247], dtype=int64)

In [87]:
import difflib

s1 = df_test.loc[3122670,'vector']
s2 = df_test.loc[3122671,'vector']

s = difflib.SequenceMatcher(None, s1, s2)

d = {'before':[], 'after': []}
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if (tag !='equal'):
        d['before'].append(s1[i1:i2])
        d['after'].append(s2[j1:j2])
print(d)


print(s.ratio())
print(s.quick_ratio())
print(s.real_quick_ratio())

{'before': [[64], [], [64, 46, 69]], 'after': [[70], [71, 72], [71, 48, 72, 73, 70]]}
0.625
0.625
0.875


In [88]:
s1

[64, 54, 65, 44, 66, 44, 45, 45, 68, 44, 64, 46, 69, 45]

In [89]:
s2

[70, 54, 65, 44, 66, 44, 71, 72, 45, 45, 68, 44, 71, 48, 72, 73, 70, 45]

In [90]:
for tag, i1, i2, j1, j2 in s.get_opcodes():
    print(tag, i1, i2, j1, j2)

replace 0 1 0 1
equal 1 6 1 6
insert 6 6 6 8
equal 6 10 8 12
replace 10 13 12 17
equal 13 14 17 18


In [108]:
df_correct = df_test[(df_test['solution_label'] == 'Correct') | (df_test['solution_label'] == 'Style') | 
                   (df_test['solution_label'] == 'Correct, unknown style')]
df_correct.reset_index(inplace=True)
df_correct

,index,step_id,status,client,uid_anon,id_anon,reply,feedback,solution_label,code,code_clean,vector
0,3122670,15944,2,web,32934,18602893,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\na = int(input())\r\n\r...,"[a, =, int, (, input, (, ), ), print, (, a, *,...","[64, 54, 65, 44, 66, 44, 45, 45, 68, 44, 64, 4..."
1,3122672,15944,2,web,30310,18604202,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Style,"# put your code here\nA = int(input(""""))\nprin...","[A, =, int, (, input, (, ``, '', ), ), print, ...","[70, 54, 65, 44, 66, 44, 71, 72, 45, 45, 68, 4..."
2,3122677,15944,2,web,27006,18605599,"{""language"": ""python3"", ""code"": ""num = int(inp...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,num = int(input())\nprint(-num)\n,"[num, =, int, (, input, (, ), ), print, (, -nu...","[79, 54, 65, 44, 66, 44, 45, 45, 68, 44, 80, 45]"
3,3122682,15944,2,web,26245,18606198,"{""language"": ""python3"", ""code"": ""integ = int(i...","{""message"": """", ""code_style"": {""quality"": {""co...",Style,integ = int(input())\nif integ < 0:\n integ...,"[integ, =, int, (, input, (, ), ), if, integ, ...","[75, 54, 65, 44, 66, 44, 45, 45, 5, 75, 53, 67..."
4,3122683,15944,2,web,33310,18608888,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\nx = int(input())\nprint(...,"[x, =, int, (, input, (, ), ), print, (, -x, )]","[82, 54, 65, 44, 66, 44, 45, 45, 68, 44, 83, 45]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2104,4622562,15944,2,web,47462,32243928,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Style,a = int(input())\nprint(a*(-1))\n,"[a, =, int, (, input, (, ), ), print, (, a, *,...","[64, 54, 65, 44, 66, 44, 45, 45, 68, 44, 64, 4..."
2105,4622563,15944,2,web,47462,32243973,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,a = int(input())\nprint(-a)\n,"[a, =, int, (, input, (, ), ), print, (, -a, )]","[64, 54, 65, 44, 66, 44, 45, 45, 68, 44, 89, 45]"
2106,4622569,15944,2,web,50966,32258309,"{""language"": ""python3"", ""code"": ""# put your co...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,# put your code here\r\nprint(-(int(input())))\n,"[print, (, -, (, int, (, input, (, ), ), ), )]","[68, 44, 48, 44, 65, 44, 66, 44, 45, 45, 45, 45]"
2107,4622570,15944,2,web,44573,32275474,"{""language"": ""python3"", ""code"": ""a = int(input...","{""message"": """", ""code_style"": {""quality"": {""co...",Correct,a = int(input())\r\nprint(a * -1)\n,"[a, =, int, (, input, (, ), ), print, (, a, *,...","[64, 54, 65, 44, 66, 44, 45, 45, 68, 44, 64, 4..."


In [93]:
codes = df_correct['vector'].values
codes

array([list([64, 54, 65, 44, 66, 44, 45, 45, 68, 44, 64, 46, 69, 45]),
       list([70, 54, 65, 44, 66, 44, 71, 72, 45, 45, 68, 44, 74, 45]),
       list([79, 54, 65, 44, 66, 44, 45, 45, 68, 44, 80, 45]), ...,
       list([68, 44, 48, 44, 65, 44, 66, 44, 45, 45, 45, 45]),
       list([64, 54, 65, 44, 66, 44, 45, 45, 68, 44, 64, 46, 69, 45]),
       list([288, 44, 66, 44, 45, 45, 668, 46, 69, 68, 44, 158, 45])],
      dtype=object)

In [99]:
import math

mse1 = []
for s1 in codes:
    sum_diff = 0  
    for s2 in codes:
        s = difflib.SequenceMatcher(None, s1, s2).ratio()
        sum_diff += s**2
    mse1.append(math.sqrt(sum_diff))
mse1

[31.855247309436137,
 28.95195255179581,
 31.79735769565579,
 16.779086538371843,
 31.508525919266578,
 31.638241416622186,
 31.875681518083837,
 31.386632877333934,
 25.156906179254516,
 25.669327716609295,
 22.581622740351705,
 31.875681518083837,
 29.13100473118589,
 32.028629966875215,
 32.028629966875215,
 31.140387657620973,
 20.776586518443064,
 20.72183020085535,
 23.264306473990317,
 26.358089905709683,
 25.45794045754184,
 25.45794045754184,
 30.935788744425608,
 30.93484836590521,
 31.937887090216343,
 25.624161821571874,
 28.473845229178206,
 25.45794045754184,
 28.473845229178206,
 30.946152958818686,
 25.45794045754184,
 26.024762151714768,
 28.473845229178206,
 24.06710035514889,
 26.179597082811934,
 26.179597082811934,
 31.070706525145876,
 21.444239434422904,
 31.638241416622186,
 25.592579608472924,
 25.47508198889445,
 29.74985628017133,
 26.179597082811934,
 31.10086171511612,
 28.473845229178206,
 28.473845229178206,
 28.473845229178206,
 26.07622192377287,
 25.45

In [103]:
import numpy as np

index_min1 = np.argmin(mse1)
index_min1

188

In [100]:
import math

mse2 = []
for s1 in codes:
    sum_diff = 0  
    for s2 in codes:
        s = difflib.SequenceMatcher(None, s1, s2).quick_ratio()
        sum_diff += s**2
    mse2.append(math.sqrt(sum_diff))
mse2

[33.967532964450264,
 30.253643431133394,
 33.11370301653029,
 19.478016858392802,
 32.85222160465211,
 33.756635699250154,
 34.008337251194064,
 32.7135339420767,
 27.662928477922787,
 27.379949617801152,
 24.100926391398207,
 34.008337251194064,
 32.339552813348,
 33.364916428022674,
 33.364916428022674,
 34.008337251194064,
 22.20739321006632,
 22.36687301265713,
 25.20243506714277,
 28.299655474622412,
 30.924850721728887,
 30.924850721728887,
 33.011947337816174,
 33.011066102393094,
 33.262119769308796,
 27.662928477922787,
 34.275280750030895,
 30.924850721728887,
 34.275280750030895,
 33.02361362727714,
 30.924850721728887,
 32.16805009118121,
 34.275280750030895,
 26.56892314908253,
 34.275280750030895,
 34.275280750030895,
 33.47818916514245,
 24.515900576337057,
 33.756635699250154,
 28.193368451462366,
 27.632130530243238,
 31.540452898697517,
 34.275280750030895,
 33.18566890574943,
 34.275280750030895,
 34.275280750030895,
 34.275280750030895,
 32.42514339785039,
 27.5094

In [104]:
index_min2 = np.argmin(mse2)
index_min2

188

In [101]:
import math

mse3 = []
for s1 in codes:
    sum_diff = 0  
    for s2 in codes:
        s = difflib.SequenceMatcher(None, s1, s2).real_quick_ratio()
        sum_diff += s**2
    mse3.append(math.sqrt(sum_diff))
mse3

[39.85610143036805,
 39.85610143036805,
 39.152247747842125,
 25.028968931269223,
 39.152247747842125,
 39.85610143036805,
 39.85610143036805,
 39.152247747842125,
 33.99492543789436,
 35.6885856522758,
 31.617482736153743,
 39.85610143036805,
 39.27679990720862,
 39.152247747842125,
 39.152247747842125,
 39.85610143036805,
 29.194073568337735,
 29.662913583766844,
 31.116900216222707,
 38.187120676724234,
 34.291411369519516,
 34.291411369519516,
 39.85610143036805,
 39.85610143036805,
 39.152247747842125,
 33.99492543789436,
 37.85289839675557,
 34.291411369519516,
 37.85289839675557,
 39.85610143036805,
 34.291411369519516,
 37.85289839675557,
 37.85289839675557,
 36.12343239123958,
 37.85289839675557,
 37.85289839675557,
 39.27679990720862,
 32.58008098179338,
 39.85610143036805,
 36.59019408024865,
 33.99492543789436,
 39.622630955783954,
 37.85289839675557,
 39.85610143036805,
 37.85289839675557,
 37.85289839675557,
 37.85289839675557,
 39.152247747842125,
 33.99492543789436,
 34

In [106]:
index_min3 = np.argmin(mse3)
index_min3

188

In [110]:
medium_solution = df_correct.iloc[index_min3]
medium_solution

index                                                       3278861
step_id                                                       15944
status                                                            2
client                                                          web
uid_anon                                                      34975
id_anon                                                    19055662
reply             {"language": "python3", "code": "# put your co...
feedback          {"message": "", "code_style": {"quality": {"co...
solution_label                                              Correct
code              # put your code here\r\nsome_int = input().str...
code_clean        [some_int, =, input, (, ), .strip, (, ), oppos...
vector            [177, 54, 66, 44, 45, 178, 44, 45, 179, 54, 71...
Name: 188, dtype: object

In [50]:
import difflib

def count_diff(s1, s2):
    d = {'before':[], 'after': []}
    seq_matcher = difflib.SequenceMatcher(None, s1, s2)
    for tag, i1, i2, j1, j2 in seq_matcher.get_opcodes():
        if (tag !='equal'):
            d['before'].append(s1[i1:i2])
            d['after'].append(s2[j1:j2])
    return d

res = []
for user in users:
    df_t = df_test[df_test['uid_anon'] == user]
    correct = df_t[(df_t['solution_label'] == 'Correct') | (df_t['solution_label'] == 'Style') | 
                   (df_t['solution_label'] == 'Correct, unknown style')].head(1)
    
    error = df_t[(df_t['solution_label'] != 'Correct') & (df_t['solution_label'] != 'Style') & 
                   (df_t['solution_label'] != 'Correct, unknown style')].head(1)
    if not correct.empty and not error.empty:
        result = error.apply(lambda x: count_diff(x['vector'], correct['vector'].values[0]), axis=1)
        res.append(result.values[0])

In [45]:
df_res = pd.DataFrame(res)
df_res

,before,after
0,"[[5, 64, 53, 67, 51, 68, 44, 64, 45, 3, 51]]",[[]]
1,"[[71, 48, 72, 73, 70]]",[[74]]
2,"[[75, 47], [76, 46], [], []]","[[81], [], [44], [45]]"
3,"[[84], [], [], [84]]","[[79], [65, 44], [45], [79]]"
4,"[[], [82], [], []]","[[82, 54, 65, 44, 66, 44, 45, 45], [44, 83, 45..."
...,...,...
666,"[[53], [85], [119, 120, 47, 90, 44, 85], [45]]","[[55], [101], [101], []]"
667,[[]],"[[44, 45]]"
668,"[[5, 79, 55, 67, 51], [30], [], [3, 51, 68, 44...","[[], [], [46, 44, 69], []]"
669,"[[64, 54], [], [5, 64, 55, 67, 51, 68, 44, 89]...","[[68, 44, 48, 44], [44], [], [], [45]]"


In [46]:
df_res['before']

0           [[5, 64, 53, 67, 51, 68, 44, 64, 45, 3, 51]]
1                                 [[71, 48, 72, 73, 70]]
2                           [[75, 47], [76, 46], [], []]
3                                   [[84], [], [], [84]]
4                                     [[], [82], [], []]
                             ...                        
666       [[53], [85], [119, 120, 47, 90, 44, 85], [45]]
667                                                 [[]]
668    [[5, 79, 55, 67, 51], [30], [], [3, 51, 68, 44...
669    [[64, 54], [], [5, 64, 55, 67, 51, 68, 44, 89]...
670                                         [[53], [91]]
Name: before, Length: 671, dtype: object

In [41]:
inv_dict = {v: k for k, v in dict_token.items()}
inv_dict

{0: 'False',
 1: 'True',
 2: 'await',
 3: 'else',
 4: 'elif',
 5: 'if',
 6: 'import',
 7: 'pass',
 8: 'None',
 9: 'break',
 10: 'except',
 11: 'in',
 12: 'raise',
 13: 'class',
 14: 'finally',
 15: 'is',
 16: 'return',
 17: 'and',
 18: 'continue',
 19: 'for',
 20: 'lambda',
 21: 'try',
 22: 'as',
 23: 'def',
 24: 'from',
 25: 'nonlocal',
 26: 'while',
 27: 'assert',
 28: 'del',
 29: 'global',
 30: 'not',
 31: 'with',
 32: 'async',
 33: 'or',
 34: 'yield',
 35: '!',
 36: '?',
 37: '_',
 38: '“',
 39: '#',
 40: '$',
 41: '%',
 42: '&',
 43: '‘',
 44: '(',
 45: ')',
 46: '*',
 47: '+',
 48: '-',
 49: '.',
 50: '/',
 51: ':',
 52: ';',
 53: '<',
 54: '=',
 55: '>',
 56: '@',
 57: '\\',
 58: '[',
 59: ']',
 60: '{',
 61: '}',
 62: '|',
 63: '~',
 64: 'a',
 65: 'int',
 66: 'input',
 67: '0',
 68: 'print',
 69: '-1',
 70: 'A',
 71: '``',
 72: "''",
 73: ',',
 74: '-A',
 75: 'integ',
 76: '2',
 77: '-integ',
 78: '-0',
 79: 'num',
 80: '-num',
 81: 'abs',
 82: 'x',
 83: '-x',
 84: 'usr',
 85: 

In [47]:
from collections import defaultdict 
dict_token = defaultdict(int)

for l in df_res['before']: 
    for change in l:
        for t in change:
            dict_token[inv_dict[t]]+=1 
for l in df_res['after']:
    for change in l:
        for t in change:
            dict_token[inv_dict[t]]+=1 
dict_token

defaultdict(int,
            {'if': 206,
             'a': 244,
             '<': 61,
             '0': 245,
             ':': 357,
             'print': 292,
             '(': 816,
             ')': 741,
             'else': 191,
             '``': 149,
             '-': 236,
             "''": 184,
             ',': 72,
             'A': 3,
             'integ': 1,
             '+': 128,
             '2': 46,
             '*': 299,
             'usr': 2,
             'x': 113,
             '>': 184,
             'number': 190,
             '-int': 22,
             'val': 3,
             'readLine': 1,
             '!': 20,
             '.toInt': 2,
             'println': 2,
             'str': 50,
             '-readLine': 1,
             'flip': 3,
             'num': 166,
             '=': 253,
             ';': 3,
             'mod.num': 1,
             'num_2': 3,
             'int': 190,
             "'-": 48,
             "'": 66,
             'pritn': 1,
             'input':

In [48]:
sorted_tuples = sorted(dict_token.items(), key=lambda item: item[1], reverse=True)
sorted_tuples

[('(', 816),
 (')', 741),
 (':', 357),
 ('*', 299),
 ('print', 292),
 ('=', 253),
 ('-1', 253),
 ('0', 245),
 ('a', 244),
 ('-', 236),
 ('if', 206),
 ('else', 191),
 ('number', 190),
 ('int', 190),
 ("''", 184),
 ('>', 184),
 ('num', 166),
 ('``', 149),
 ('+', 128),
 ('x', 113),
 ('input', 78),
 ('abs', 73),
 (',', 72),
 ('n', 69),
 ("'", 66),
 ('<', 61),
 ('str', 50),
 ("'-", 48),
 ('2', 46),
 ('integer', 43),
 ('-a', 42),
 ('-num', 40),
 ('{', 32),
 ('}', 32),
 ('==', 30),
 ('b', 30),
 ('-number', 27),
 ('1', 26),
 ('-int', 22),
 ('f', 22),
 (']', 21),
 ('!', 20),
 ('[', 20),
 ('-x', 19),
 ('user_input', 17),
 ('not', 15),
 ('elif', 15),
 ('value', 14),
 ('i', 13),
 ('return', 12),
 ('-n', 12),
 ('-abs', 11),
 ('y', 9),
 ('result', 9),
 ('567', 8),
 ('c', 8),
 ('def', 7),
 ('-2', 7),
 ('-i', 6),
 ('num_1', 6),
 ('-integer', 6),
 ("f'-", 6),
 ('user_number', 5),
 ('+a', 5),
 ('in', 5),
 ('is', 4),
 ('opposite', 4),
 ('negate', 4),
 ('-567', 4),
 ("'+", 4),
 ('nummer', 4),
 ('import', 